In [1]:
#@title Connect to Google drive { display-mode: "form" }

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# prompt: create a function that pulls a git repo, with the paramater 'repo'

import os
import subprocess
import urllib.parse


def install_pip_if_needed(package, version=None):
  """
  Installs a package if it is not already installed.

  Args:
    package: The name of the package to install.
    version: The version of the package to install (optional).
  """

  try:
    if version is not None:
      subprocess.run(["pip", "show", f"{package}=={version}"], check=True)
    else:
      subprocess.run(["pip", "show", package], check=True)
  except subprocess.CalledProcessError:
    # Package is not installed, install it
    if version is not None:
      subprocess.run(["pip", "install", f"{package}=={version}"], check=True)
    else:
      subprocess.run(["pip", "install", package], check=True)


#prompt: create function to install a package with apt,
#         with an option version number as a parameter
def install_package_apt(package_name, version=None):
  """
  Installs a package using apt.

  Args:
    package_name: The name of the package to install.
    version: The version of the package to install (optional).

  Example:
    install_package_apt("cuda-toolkit-11-8")  # Install the latest version
    install_package_apt("cuda-toolkit-11-8", "11.8.0")  # Install a specific version
  """

  if version is not None:
    command = f"apt install -y {package_name}={version}"
  else:
    command = f"apt install -y {package_name}"

  subprocess.run(command, shell=True, check=True)
  print(f"{package_name} installed successfully.")


#prompt: create function to get the last path of a url, with the paramater 'url'
def get_last_path_of_url(url):
  """
  Gets the last path of a URL.

  Args:
    url: The URL.

  Returns:
    The last path of the URL.
  """

  # Strip off any trailing slashes from the URL
  url = url.rstrip('/')
  parsed_url = urllib.parse.urlparse(url)
  print(f"Last path of {url} is {parsed_url.path.split('/')[-1]}")
  return parsed_url.path.split('/')[-1]


# prompt: create function to clone a repo if it does not already exist. if it does exist, pull the repo. parameter name is repo
def clone_or_pull_repo(repo):
  """
  Clones a git repo if it doesn't exist, otherwise pulls the repo.

  Args:
    repo: The URL of the git repo.

  Returns:
    None
  """
  if repo == "":
    return

  # Get the last path of the URL.
  newdir=get_last_path_of_url(repo)
  print(f"Repo directory is:{newdir}")
  # Check if the repo exists.
  if not os.path.exists(newdir):
    # Clone the repo.
    print(f"Cloning {repo}...")
    !git clone {repo}
  else:
    # Pull the repo.
    os.chdir(newdir)
    print(f"Pulling {repo}...")
    !git pull

def packages():
  '''
  Returns a list of installed packages.
  '''
  import sys, subprocess
  return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  '''
  Downloads a model from a URL.
  '''
  if 'huggingface.co' in url:
    print(f"Downloading {url}...")
    filename = url.split('/')[-1]
    filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    print(f"Downloading from civitai: {url}...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}



In [4]:


#@title Setup 2 (pull repos) { display-mode: "form" }
#@markdown 1. Clones repo
#@markdown 2. Cd's into repo directory
#@markdown
#@markdown Root path:
root = "/content" #@param {type:"string"}
install_dir = "forge" #@param {type:"string"}
launch_dir = "stable-diffusion-webui-forge" #@param {type:"string"}
#@markdown *APT installs*
#@markdown
aptpackage = [""]*6
git_repo = [""]*6
model = [""]*6

#@markdown Fill in APT packages to install
aptpackage[0] = "aria2" #@param {type:"string"}
aptpackage[1] = "ffmpeg" #@param {type:"string"}
aptpackage[2] = "libvulkan1" #@param {type:"string"}
aptpackage[3] = "" #@param {type:"string"}
aptpackage[4] = "" #@param {type:"string"}
aptpackage[5] = "" #@param {type:"string"}
#@markdown Fill in repositories to pull (urls)
#@markdown   *example: https://github.com/zixaphir/Stable-Diffusion-Webui-Civitai-Helper.git*
git_repo[0] = "https://github.com/lllyasviel/stable-diffusion-webui-forge.git" #@param {type:"string"}
git_repo[1] = "" #@param {type:"string"}
git_repo[2] = "" #@param {type:"string"}

#@markdown *Models to download*
model[0] = "https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt" #@param {type:"string"}
model[1] = "https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_pruned.safetensors" #@param {type:"string"}
model[2] = "https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt" #@param {type:"string"}
model[3] = "https://huggingface.co/Lykon/DreamShaper/resolve/main/DreamShaper_8_INPAINTING.inpainting.safetensors" #@param {type:"string"}
model[4] = "https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt" #@param {type:"string"}
model[5]  ="" #@param {type:"string"}

#@markdown *Extensions*
extension = [""]*6
extension[0] = "https://github.com/continue-revolution/sd-forge-animatediff" #@param {type:"string"}
extension[1] = "https://github.com/zanllp/sd-webui-infinite-image-browsing" #@param {type:"string"}
extension[2] = "https://github.com/BlafKing/sd-civitai-browser-plus" #@param {type:"string"}
extension[3] = "https://github.com/fkunn1326/openpose-editor" #@param {type:"string"}
extension[4] = "https://github.com/Bing-su/adetailer" #@param {type:"string"}
extension[5] = "https://github.com/deforum-art/sd-forge-deforum" #@param {type:"string"}

#start setup
!apt update
!apt upgrade

!mkdir -p {root}/{install_dir}
os.chdir(root)

#install packages
for package in aptpackage:
  if package != "":
    install_package_apt(package)

#upgrade packages
#--upgrade numpy==1.23.5

install_pip_if_needed('numpy','1.23.5')
pip_packages = [
    "pyngrok",
    "insightface",
    "gradio",
    "pytorch-lightning",
    "torchmetrics",
    "torch-fidelity",
    "pydub",
    "ffmpeg",
    "ffmpeg-python",
    "torchvision",
    "torch",
    "torchaudio",
    "huggingface-hub",
    "bark",
]


# Loop through the array and install each package
for package in pip_packages:
  print(f"Installing pip {package}...")
  install_pip_if_needed(package)

os.chdir(root+'/'+install_dir)
#pull repos
for repo in git_repo:
  if repo != "":
    print(f"Cloning {repo}...")
    clone_or_pull_repo(repo)

print("Checking model downloads...")
os.chdir(root+'/'+install_dir+'/'+launch_dir+'/models/Stable-diffusion')
for model in model:
  if model != "":
    print(f"Downloading model {model}...")
    downloadModel(model)

#Extensions are git clones into the extensions directory
print("downloading extensions...")
os.chdir(root+'/'+install_dir+'/'+launch_dir+'/extensions')
for extension in extension:
  if extension != "":
    print(f"Downloading extension {extension}...")
    clone_or_pull_repo(extension)


print("Done with setup!")
#end setup

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,352 kB]
Get:13 http://security.ubunt

In [5]:
#@title Launching { display-mode: "form" }
username = '' #@param {type:"string"}
#@markdown Default: "`user`"<br>
#@markdown Sets the username for the webui

#@markdown

password = '' #@param {type:"string"}
#@markdown Default: "`pass`", **please change this**<br>
#@markdown Sets the password for the webui

#@markdown

args = '--theme dark' #@param {type:"string"}
#@markdown Default: empty<br>
#@markdown Sets the other args for the webui

#@markdown
launch_dir = "stable-diffusion-webui-forge"
os.chdir(f'{root}/{install_dir}/{launch_dir}')
args = f' {args} --gradio-queue --enable-insecure-extension-access --share '
!python launch.py $args

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version: f0.0.17v1.8.0rc-latest-276-g29be1da7
Commit hash: 29be1da7cf2b5dccfc70fbdd33eb35c56a31ffb7
Installing clip
Installing open_clip
Cloning assets into /content/forge/stable-diffusion-webui-forge/repositories/stable-diffusion-webui-assets...
Cloning into '/content/forge/stable-diffusion-webui-forge/repositories/stable-diffusion-webui-assets'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 0), reused 20 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 132.70 KiB | 22.12 MiB/s, done.
Cloning Stable Diffusion into /content/forge/stable-diffusion-webui-forge/repositories/stable-diffusion-stability-ai...
Cloning into '/content/forge/stable-diffusion-webui-forge/repositories/stable-diffusion-stability-ai'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (357/357), done.
remote: Compres